In [ ]:
 import os
 files = os.listdir('/content/drive/MyDrive/Colab datasets/weatherdata2')
 station_Name=[x.split('.')[0] for x in files]
 print(files, )
 print(station_Name)

['OAK_PARK.csv', 'SHANNON_AIRPORT.csv', 'ATHENRY.csv', 'BELMULLET.csv', 'CASEMENT.csv', 'CLAREMORRIS.csv', 'CORK_AIRPORT.csv', 'DUBLIN_AIRPORT.csv', 'FINNER.csv', 'JOHNSTOWN.csv', 'KNOCK_AIRPORT.csv', 'MALIN_HEAD.csv', 'MOORE_PARK.csv', 'MT_DILLON.csv', 'MULLINGAR.csv', 'Newport.csv', 'RochesPoint.csv', 'Sherk_inIsland.csv', 'VALENTIA_OBSERVATORY.csv', 'Markree.csv', 'BALLYHAISE.csv', 'Dunsany.csv', 'Gurteen.csv', 'Final_Xdataset_v3.csv']
['OAK_PARK', 'SHANNON_AIRPORT', 'ATHENRY', 'BELMULLET', 'CASEMENT', 'CLAREMORRIS', 'CORK_AIRPORT', 'DUBLIN_AIRPORT', 'FINNER', 'JOHNSTOWN', 'KNOCK_AIRPORT', 'MALIN_HEAD', 'MOORE_PARK', 'MT_DILLON', 'MULLINGAR', 'Newport', 'RochesPoint', 'Sherk_inIsland', 'VALENTIA_OBSERVATORY', 'Markree', 'BALLYHAISE', 'Dunsany', 'Gurteen', 'Final_Xdataset_v3']


In [ ]:
#Run Once
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz
!tar xf spark-3.1.1-bin-hadoop2.7.tgz
!pip install -q findspark
#Run Once
import os
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()
spark

In [ ]:
import pandas as pd
from pyspark.sql.functions import to_timestamp
from pyspark.sql.types import *
from pyspark.sql import functions as F
sc = spark.sparkContext
from pyspark.sql import Row
from pyspark.sql.functions import isnull, when, count, col
from pyspark.sql.functions import udf


In [ ]:
labels = [
     ('date',StringType()),
     ('ind1',DoubleType()),
     ('rain',DoubleType()),
     ('ind3',DoubleType()),
     ('temp',DoubleType()),
     ('ind5',DoubleType()),
     ('wetb',DoubleType()),
     ('dewpt',IntegerType()),
     ('vappr',IntegerType()),
     ('rhum', IntegerType()),
     ('msl',DoubleType()),
     ('ind11',DoubleType()),
     ('wdsp',IntegerType()),
     ('ind13',DoubleType()),
     ('wddir',IntegerType())
]

In [ ]:
# Creating the schema that will be passed when reading the csv
schema = StructType([StructField (x[0], x[1], True) for x in labels])


In [ ]:
maindf = spark.createDataFrame(
    [
        (1, "2011-01-01 00:00:00"),  # create your data here, be consistent in the types.
    ],
    ["id", "date"]  # add your column names here
)
maindf.printSchema()
maindf.show()

root
 |-- id: long (nullable = true)
 |-- date: string (nullable = true)

+---+-------------------+
| id|               date|
+---+-------------------+
|  1|2011-01-01 00:00:00|
+---+-------------------+



In [ ]:
maindf.show()

+---+-------------------+
| id|               date|
+---+-------------------+
|  1|2011-01-01 00:00:00|
+---+-------------------+



In [ ]:
 import os
 files = os.listdir('/content/drive/MyDrive/Colab datasets/weatherdata2')
 station_Name=[x.split('.')[0] for x in files]
 print(files, )
 print(station_Name)

for i in station_Name:
  # get name of station
  name = i

# get each file and apply scema
  df1  = spark.read.csv('/content/drive/MyDrive/Colab datasets/weatherdata2/{}.csv'.format(i), header=True, sep=",", schema=schema )

# convertstring to timestamp
  df2 = df1.withColumn("TS_date", to_timestamp(df1["date"], 'dd/MM/yyyy HH:mm'))
  df3 = df2.drop('date')

# get data between 2011-01-01 00:00:00 and 2021-01-01 00:00:00
  finalDF12 = df3.select('TS_date', "rain",'temp','rhum', 'msl', 'wdsp', 'wddir').filter((df3['TS_date'] >= "2011-01-01 00:00:00") & 
                                                                                       (df3['TS_date'] <= "2021-01-01 00:00:00"))
  
# renames the column labels with postfix station name
  finalDF13 = finalDF12.selectExpr("TS_date as date","rain as rain_{}".format(name), "temp as temp_{}".format(name), "rhum as rhum_{}".format(name), \
                                 "msl as msl_{}".format(name), "wdsp as wdsp_{}".format(name), "wddir as wddir_{}".format(name)
# check for nulls and replace with 0
  finalDF13.show()
  finalDF13.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in finalDF13.columns]).show()
  finalDF13 = finalDF13.na.fill(value=0)
  finalDF13.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in finalDF13.columns]).show()
# preform an outer join to mainf to append each weather station Dataframe to it.
  maindf = maindf.join(finalDF13_with_cat, on=['date'], how='outer')
# show the final datframe. 
maindf.show(truncate=False)

+-------------------+-------------+-------------+-------------+------------+-------------+--------------+
|               date|rain_OAK_PARK|temp_OAK_PARK|rhum_OAK_PARK|msl_OAK_PARK|wdsp_OAK_PARK|wddir_OAK_PARK|
+-------------------+-------------+-------------+-------------+------------+-------------+--------------+
|2011-01-01 00:00:00|          0.0|          4.5|           96|      1028.3|            1|           280|
|2011-01-01 01:00:00|          0.0|          4.6|           96|      1028.2|            1|           350|
|2011-01-01 02:00:00|          0.0|          4.4|           96|      1028.2|            2|            20|
|2011-01-01 03:00:00|          0.0|          4.3|           97|      1028.0|            1|           210|
|2011-01-01 04:00:00|          0.0|          4.5|           96|      1027.8|            1|           240|
|2011-01-01 05:00:00|          0.0|          4.6|           97|      1027.8|            1|           290|
|2011-01-01 06:00:00|          0.0|          4

In [ ]:
maindf.show(truncate=False)

+-------------------+----+-------------+-------------+-------------+------------+-------------+--------------+--------------------+--------------------+--------------------+-------------------+--------------------+---------------------+------------+------------+------------+-----------+------------+-------------+--------------+--------------+--------------+-------------+--------------+---------------+-------------+-------------+-------------+------------+-------------+--------------+----------------+----------------+----------------+---------------+----------------+-----------------+-----------------+-----------------+-----------------+----------------+-----------------+------------------+-------------------+-------------------+-------------------+------------------+-------------------+--------------------+-------------+-------------+-------------+------------+-------------+--------------+--------------------+--------------------+--------------------+-------------------+---------------

In [ ]:
# this will right it to a folder but the csv will be in partions in colab about 200 seperate csv files. 
#maindf.write.csv('/content/drive/MyDrive/Colab datasets/weatherdata2/finaldataset.csv')

In [ ]:
# get the dimensions of the data
(maindf.count() , len(maindf.columns))

(87673, 146)

In [ ]:
# this uses pandas to create 1 csv file. 
maindf.toPandas().to_csv('/content/drive/MyDrive/Colab datasets/Final/Final_Xdataset_v3.csv')